## Multi-criteria Objective Function

### MIP Problem Definition
* Sets: J (Orders/Missions), $I_{max}$ (Max number of operators), T (Order types).
Note that J can be expanded in $J' = J \cup \{\text{0}\}$ in order to have the initial base virual node.
* Variables: $x_{ij}$ (Assignment), $S_{j}$ (Start Time), ${Cj}$ (Completion Time), $y_{i}$ (Operator Activation), $S_{i, first}$(Operator Start Time), $C_{i, last}$(Operator Completion Time), Z (Makespan)	  
Sequencing variable: $z_{ikj}$ (operator i goes from order k to order j)
* Parameters:
  - $P_{ij}$(Processing time) => time of loading and unloading of pallet j.
  - $T{jk}$ (Travel time from j to k) => calculated basing on any possible.combination between two orders i and k.
  - $S{it}$ (Skill score) => the score that a fork_lift is adequate to pallet dimensions "less loss in fork space".
  - $O{jt}$ (Order type) => could be ignored in the future.
  - $H_{fixed}$ (Work hours per operator/machine)
  - M (Big number)

*Note that all time units should be standarized in minutes {shift time, operator speed that affect the processing time, ...}

#### Constaints and Objective function

1. Each order j is assigned to a single operator i
$$\sum_{i \in I{max}} x_{i j} = 1 \quad \forall j \in J$$

2. The type of order j should be handled by an operator i that has skill score >= 1 for such order
 $$\sum_{t \in T} O_{j t} \cdot S_{i, t} \ge x_{i j} \quad \forall i \in I_{max}, j \in J$$

3. The linking between order k and order j over the same operator i
 $$\sum_{k \in J \cup \{\text{Base}\}} z_{i k j} = x_{i j} \quad \forall i \in I_{max}, j \in J$$
(The operator i must travel to j if and only if i is assigned to j.)

4. the number of flow-ins (j->k) should be equal to the number flow-outs (l->j) of the same operator i for any order j
$$\sum_{k \in J \cup \{\text{Base}\}} z_{i j k} = \sum_{l \in J \cup \{\text{Base}\}} z_{i l j} \quad \forall i \in I_{max}, j \in J \text{ and } k \ne l$$
(Including both directions towards the base)

5. Time sequencing (the start time of order k is after the completion time of the previous linked order j + the travel time to reach order k)
$$S_k \ge C_j + T_{j k} - M \cdot (1 - z_{i j k}) \quad \forall i \in I_{max}, j, k \in J, j \ne k$$
(M is a big number that activate the constraint in case of linking order k and order j; otherwise, the constraint give a very large negative number)

6. The completion time of order j is the time to start it + the processing time w.r.t. the assigned operator i
$$C_j = S_j + \sum_{i \in I} x_{i, j} \cdot P_{i j} \quad \forall j \in J, i \in I_{max}$$
7. Common MTZ (Miller-Tucker-Zemlin) constraints to prevent isolated loops/subtours
 $$u_j - u_k + |J| \cdot z_{i j k} \le |J| - 1 \quad \forall i \in I_{max}; j, k \in J, j \ne k$$
(Where $u_{j}$ and $u_{k}$ are helper variables representing the sequence position of j and k on the route,   
and $2 \le u_{j} \le |J|+1$.)

8. An order j can only be assigned to operator i if and only if the operator i is activated $(y_{i}=1)$.
$$x_{i j} \le y_{i} \forall i \in I_{max},j \in J$$

9. The time elapsed between an operator's first assigned task and his last assigned task must not exceed the fixed shift length $H_{fixed}$.
$$C_{i,last} - S_{i,first} \le H_{fixed}⋅y_{i} \forall i \in I_{max}$$

      - $C_{i,last}$: The completion time of last task per operator i.

      - $S_{i,first}$: The start time of first task per operator i.

      - Note that: $P_{i,0}=0$, $S_{0}=0, C_{0}=0$

      - (This requires additional constraints to define $C_{i,last}$ and $S_{i,first}$ based on the $z_{ijk}$ sequencing variables and a virtual Base node):
        * $S_{i,first}=S_{k}$ $\text{if } z_{i,0,k}=1$
        * $C_{i,last}=C_{j}+T_{j,0}$ $\text{if } z_{i,j,0}=1$

10. The total number of trips that end at the Base (k=0) must equal the number of operators who are activated ($y_{i}=1$).
$$\sum_{j \in J} z_{ij0} =y_{i} \forall i \in I_{max}$$

11. Objective function (multi-objective): minimize the overall time to complete all orders (Makespan) AND minimize the fixed cost of labor (the number of operators used).
$$\text{Minimize } Z=(\alpha⋅Z)+(\beta⋅ \sum_{i \in I_{max}}y_{i​})$$

    - Z: The Makespan (maximum completion time, costed by α).

    - $\sum_{i\in I_{max}}y_{i​}$: The total number of operators used (costed by β).

    - $\alpha, \beta$: Weighting factors you choose to define the trade-off. (α high = minimize time; β high = minimize staff).

    - $I_{max}$: The set of all available operators we could potentially use (unactivated operators).

*Base order/mission is at position [0, 0], from which all operator should start the day.

### GNN Problem Definition

The graph $\mathcal{G}=(\mathcal{V}, \mathcal{E})$ must capture all orders, all potential resources, and all spatial relationships necessary for the GNN to make an informed, multi-objective decision.

* Nodes ($\mathcal{V}$)
  - Order Nodes $J$ (up to 10,000): Initial Features ($h_v$) => Location coordinates $(x_j, y_j)$, Required Skill ID, Processing Time ($P_{ij}$).
  - Operator Nodes $I_{max}$ (Pool): Initial Features ($h_v$) => Current Location, Operator Skill Set, Fixed Shift Capacity ($H_{fixed}$), Status (Idle/Busy).

* Edges ($\mathcal{E}$)
  - Order-Order ($\boldsymbol{e_{jk}}$) featured with "Travel Time $T_{jk}$": Represents the static distance map of the warehouse (ex. calculated with A*).
  - Operator-Order ($\boldsymbol{e_{ij}}$) featured with current "Travel Time $T_{i, j}$": Represents the dynamic accessibility of $\text{Order}_j$ by $\text{Operator}_i$ from their current position.



### GNN Architecture
* Message Passing Layers: These layers iteratively compute a contextual node embedding for every order and operator by aggregating information from their neighbors. This allows the GNN to understand complex trade-offs, such as: "This order is close to Operator A, but Operator B is already assigned nearby tasks, making Operator B's route more efficient."B.

  * Dual Processing (Hidden Layers): The GNN's deep Message Passing layers are trained to understand the complex relationship between scheduling efficiency and labor cost.
    - Layer 1: Assignment Policy ($P_{assign}(i, j)$): This requires a set of weights ($W_{assign}$) and biases ($b_{assign}$) to map the GNN's final node embeddings to the assignment scores. This is a large layer, as it covers all $I_{max} \times J$ potential assignments.
    - Layer 2: Activation Policy ($P_{activate}(i)$): This requires a separate set of weights ($W_{activate}$) and biases ($b_{activate}$) to map the operator node embeddings to the activation scores ($y_i$). This layer must learn a distinct function specific to resource capacity.

    During this phase, the network learns features relevant to both:
    - Resource Features: Embeddings learn if an operator is "cost-effective" to activate (reflecting the $\beta$ penalty).
    - Scheduling Features: Embeddings learn the optimal sequence and travel time (reflecting the $\alpha$ penalty).

* Single Output Layer (Assignment Policy $\pi$):
 The final layer outputs only the Assignment Score ($P_{assign}(i, j)$) for every possible $\text{Operator}_i$ to $\text{Order}_j$ pair. This score reflects the GNN's learned opinion on whether that assignment is globally optimal.
 Operators with no associated assignments (missions) are considered "unactivated".

 * Final Weighted Loss ($\mathcal{L}_{\text{Total}}$):  
 The final loss value, which is minimized during training, is the weighted combination of these two aggregated errors:$$\mathcal{L}_{\text{Total}} = \left( \alpha \cdot \mathcal{L}_{\text{Scheduling}} \right) + \left( \beta \cdot \mathcal{L}_{\text{Activation}} \right)$$

    - Total Activation LossThe total activation error is the sum of local errors across all potential operator nodes:$$L_{{Activation}} = \sum_{i \in I_{\text{max}}} \text{BCE}(\hat{y}_i \text{ vs } y_i^*)$$
    - Total Scheduling LossThe total scheduling error is the sum of local errors across all potential sequencing flows (edges):$$L_{\text{Scheduling}} = \sum_{i \in I_{\text{max}}} \sum_{j, k \in J,  j \ne k} \text{BCE}(z_{ijk} \text{ vs } z_{ijk}^*)$$

    This process ensures that every decision point in the system—whether it's the strategic choice to activate an operator ($L_{\text{Activation}}$) or the tactical choice of sequence ($L_{\text{Scheduling}}$), contributes to the single error signal used to update the GNN's parameters.